# Exam Fear Problem

Lab assignment 

In [96]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination
import numpy as np

# First Set the structure


In [97]:
exam_fear = BayesianModel([("B","A"),("B","R"),("A","S"),("R","S")])


# Setup the relationship(CPDs)

In [98]:
boring_cpd = TabularCPD(variable='B',variable_card = 2, values=[[.50], [.50]])


In [99]:
revise_cpd = TabularCPD(variable='R',variable_card= 2, values=[[.2,.7], [.8,.3]],evidence=['B'], evidence_card=[2])

In [100]:
attend_cpd = TabularCPD(variable='A',variable_card= 2,values=[[.5,.9], [.5,.1]],evidence=['B'], evidence_card=[2])

In [101]:
scare_cpd = TabularCPD(variable='S',variable_card= 2, values=[[0, .4, .2, 1],
                                         [1, .6, .8, 0]],evidence=['R', 'A'], evidence_card=[2, 2])

# add relationship of model

In [102]:
exam_fear.add_cpds(boring_cpd,revise_cpd,attend_cpd,scare_cpd)

In [103]:
"""The network structure of a Graphical Model encodes the inde-
pendence conditions between the random variables. pgmpy also
has methods to determine the local independencies, D-Separation,
converting to a markov model etc. A few example are shown
below:"""

'The network structure of a Graphical Model encodes the inde-\npendence conditions between the random variables. pgmpy also\nhas methods to determine the local independencies, D-Separation,\nconverting to a markov model etc. A few example are shown\nbelow:'

In [104]:
#examine the structure of graph
exam_fear.get_cpds()

[<TabularCPD representing P(B:2) at 0x7f33f63c4860>,
 <TabularCPD representing P(R:2 | B:2) at 0x7f33f63c4908>,
 <TabularCPD representing P(A:2 | B:2) at 0x7f33f63c4978>,
 <TabularCPD representing P(S:2 | R:2, A:2) at 0x7f33f63c4b38>]

In [105]:
exam_fear.check_model()


True

# active trail nodes

In [106]:
exam_fear.active_trail_nodes("B")

{'B': {'A', 'B', 'R', 'S'}}

# Find local independence

In [107]:
exam_fear.local_independencies("B")
exam_fear.local_independencies("A")
exam_fear.local_independencies("R")

(R _|_ A | B)

# Get all independencies

In [108]:
exam_fear.get_independencies()

(B _|_ S | A, R)
(A _|_ R | B)
(R _|_ A | B)
(S _|_ B | A, R)

# Making Inference

In [109]:
fear_infer = VariableElimination(exam_fear)


In [110]:
prob_scare=fear_infer.query(variables=["S"])
print(prob_scare)

Eliminating: R: 100%|██████████| 3/3 [00:00<00:00, 380.17it/s]

+------+----------+
| S    |   phi(S) |
+======+==========+
| S(0) |   0.3160 |
+------+----------+
| S(1) |   0.6840 |
+------+----------+


# Problem-1

In [111]:
#Suppose we know that the course was boring or not boring, and want to work
#out how likely it is that we will scare before exam.

In [112]:
prob_S_B = fear_infer.query(variables=["S"],evidence={"B":1})
print("S|B",prob_S_B)

Eliminating: R: 100%|██████████| 2/2 [00:00<00:00, 723.47it/s]

S|B +------+----------+
| S    |   phi(S) |
+======+==========+
| S(0) |   0.1120 |
+------+----------+
| S(1) |   0.8880 |
+------+----------+


In [113]:
"""inference: if boring then scared 89% approx, if not boring then scared 11% approx."""

'inference: if boring then scared 89% approx, if not boring then scared 11% approx.'

# Problem-2

In [114]:
prob_R_S = fear_infer.query(variables=['R'], evidence={'S': 1})
print("R| S",prob_R_S)

Eliminating: B: 100%|██████████| 2/2 [00:00<00:00, 538.21it/s]

R| S +------+----------+
| R    |   phi(R) |
+======+==========+
| R(0) |   0.6082 |
+------+----------+
| R(1) |   0.3918 |
+------+----------+


In [115]:
prob_A_S = fear_infer.query(variables=["A"],evidence={"S":1})
print("A|S",prob_A_S)

Eliminating: B: 100%|██████████| 2/2 [00:00<00:00, 591.16it/s]

A|S +------+----------+
| A    |   phi(A) |
+======+==========+
| A(0) |   0.9254 |
+------+----------+
| A(1) |   0.0746 |
+------+----------+


In [116]:
"""Inference: Hence, I am scared because of not revising the chapters."""

'Inference: Hence, I am scared because of not revising the chapters.'